## Get Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
print('tf-version: ', tf.__version__)

# my plot function for confusion matrix
def plot_cm(mat):
  classes = np.arange(cm.shape[0])
  plt.imshow(mat, cmap=plt.cm.Blues)
  for (j,i),label in np.ndenumerate(mat):
    plt.text(i,j,np.round(label,2),ha='center',va='center')

  plt.colorbar()
  plt.title('Confusion Matrix')
  plt.xlabel('True label')
  plt.ylabel('Pred label')
  plt.xticks(classes)
  plt.yticks(classes)
  plt.show()

# Convolution Neural Network

## Challenges from previous work (MNIST)






<div>
   <img src="https://github.com/thomasmanke/ABS/raw/main/figures/smileys.png",  width="400">
</div>





- training (and test) data were highly structured: 
- fixed size
- grey scale
- item centered
- only single item 

**Discussion**: 
Other possible challenges?
- ...
- ...
-

Algorithms should be:

- robuts to those changes within one class (e.g. cat)
- generic and transferable to all other classes 
- interpretable

## A short history

Algorithms, Compute Power, Data, Data & Data

- 1958 Rosenblatt: The perceptron: A probabilistic Model for Information Storage and  Organization in the Brain.
- 1998 Le Cun et al.  (MNIST):  60000 images of 10 handwritten digits ($10^7$ pixels) + CPU ($10^6$ transitors)
- 2012 Alex Krizhevsky et al (ImageNet): 1.3M images for 1000 classes ($10^{14}$ pixels) + GPU ($10^9$ transistors)
- 2021 Yang et al. (MedMNIST): 700k images for 2-11 classes 
https://github.com/MedMNIST/MedMNIST
- 2022 Google Open Images v6: 60M images, 20000 classes: https://storage.googleapis.com/openimages/web/index.html



## Another classical dataset: CIFAR-10

This is a set of 50k images in 10 categories.
They are rather coarse (32 x 32), but unlike MNIST (Handwritten Digits) they are not as standardized.



In [ ]:
# cell takes 1-2 minutes (on my regular home network)
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# normalization
X_train, X_test = X_train / 255.0, X_test / 255.0

# just for easier reference to replace integers with names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Group Task (40 min): Explore, Model, Fit, Evaluate

## Explore

## A Simple Neural Network

... you might copy one from handwritten digits

## Fit & Evaluate

## Report back and Discussion (15 min)

# CNN: A picture and some jargon

![CNN_convlayer](https://upload.wikimedia.org/wikipedia/commons/6/68/Conv_layer.png)
(from wikipedia.org)

### Filters

![filter](https://wiki.tum.de/download/attachments/23572254/cnn6.png)
(from wiki.tum.de)

- input layer: image shape [w,h,3]
- convolutional layer (Conv): filters

  - detect pattern (e.g. horizontal, vertical, diagonal lines)
  - have the same depth as input: 28x28x3 --> 5x5x6 
  - several filter per layer: different filters applied to same spatial location in image

- pooling filter (Pool): spatially downsampling, depth stay the same (e.g. max or average)

Lower layers: Primitive concepts

Higher layers: Higher order concepts (reasoning on top of edge maps)


## CNN: tensorflow implementation

In [ ]:
nc = np.unique(y_train).size  # number of classes / labels in training set
l_name = 'sparse_categorical_crossentropy'
a_name = 'sparse_categorical_accuracy'

input_shape = X_train.shape[1:]

print('X_train.shape:     ', X_train.shape)
print('input_shape:       ', input_shape)
print('number of classes: ', nc)

model = tf.keras.models.Sequential(name='CNN')

# Convolutional layers
model.add(tf.keras.layers.InputLayer(input_shape))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', name='first_conv'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

#model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(tf.keras.layers.MaxPooling2D((2, 2)))
#model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))

# as before
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu', name='last'))
model.add(tf.keras.layers.Dense(nc, activation='softmax', name='output'))

model.compile(optimizer='adam', loss=l_name, metrics=a_name)
model.summary()

## CNN: fitting and time for some theory

The following cell will fit the model. This will take some time - especially without dedicated hardware (e.g. GPU) or further optimization (improved algorithm).

Please start the cell. While it runs I will provide some explanations what is going on under the hood.

In [ ]:
fh = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

plt.plot(fh.history['loss'])
plt.plot(fh.history['val_loss'])
plt.show()

## Saving the model

In [ ]:
# Save the model. Choose suitable path and name. 
model.save('my_cifar10_model')

# Load the model (use latter)
model = tf.keras.models.load_model("my_cifar10_model")

## Evaluations

In [ ]:
eval = model.evaluate(X_test,  y_test)
print('evaluation ',eval)
test_acc = eval[1]

## Plotting history and test accuracy
plt.plot(fh.history['accuracy'], label='train accuracy')
plt.plot(fh.history['val_accuracy'], label = 'valid accuracy')
plt.axhline(y=test_acc, color='green', linestyle='-.',label = 'test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='upper left')
plt.show()

pred = model.predict(X_test)

y_pred = np.argmax(pred, axis=1)
cm=confusion_matrix(y_pred, y_test)
plot_cm(cm)

**Message:** A clear improvement over the previous network. Possibly some signs of overfitting.

**Discussion:** What could be further improvements?

## Predictions

**Task:** Explore some predictions on the test data

In [ ]:
idx=4
X = X_test[idx]
X = np.expand_dims(X, axis=0)

pred = model.predict(X)
i_max=np.argmax(pred)

plt.figure(figsize=(12,6))
ax = plt.subplot(2,2,1)
plt.imshow(X_test[idx]) 

ax = plt.subplot(2,2,2)
plt.bar(range(10), pred[0])
plt.title(class_names[i_max])
plt.show()